In [585]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tts
from sklearn import metrics
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
import h2o
from h2o.automl import H2OAutoML

In [596]:
df= pd.read_csv('train.csv')

In [597]:
df=df.set_index('id')

In [598]:
df.sample(10)

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
15018,0.33,Good,D,SI1,63.1,55.0,4.42,4.46,2.80,567
19435,0.75,Ideal,F,SI1,61.8,56.0,5.84,5.87,3.62,3336
13574,0.50,Good,F,SI2,57.7,62.0,5.22,5.18,3.00,926
19795,1.33,Ideal,J,VS1,61.3,57.0,7.08,7.11,4.35,5852
39322,0.51,Ideal,I,SI2,61.3,55.0,5.18,5.19,3.18,991
31683,0.59,Ideal,E,SI1,61.4,56.0,5.36,5.39,3.30,1664
17885,0.31,Very Good,E,SI1,63.0,59.0,4.26,4.31,2.70,544
33547,1.15,Ideal,G,VVS2,61.8,56.0,6.72,6.75,4.16,9457
33210,1.04,Premium,E,VS2,61.1,59.0,6.54,6.56,4.00,7047


In [599]:
df.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [600]:
corr=df.corr()

In [601]:
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1') 
 .set_precision(2))

<ipython-input-601-2f759844c1a2>:4: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  (corr


,carat,depth,table,x,y,z,price
carat,nan,nan,nan,nan,nan,nan,nan
depth,0.03,nan,nan,nan,nan,nan,nan
table,0.18,-0.30,nan,nan,nan,nan,nan
x,0.98,-0.03,0.20,nan,nan,nan,nan
y,0.97,-0.03,0.19,0.99,nan,nan,nan
z,0.97,0.09,0.15,0.99,0.98,nan,nan
price,0.92,-0.02,0.13,0.88,0.88,0.88,nan


In [602]:
df.clarity.value_counts()

SI1     9746
VS2     9175
SI2     6912
VS1     6137
VVS2    3822
VVS1    2732
IF      1367
I1       564
Name: clarity, dtype: int64

In [603]:
def clarity(x):
    if x=='I1':
        return 1
    if x=='SI2':
        return 2
    if x=='SI1':
        return 3
    if x=='VS2':
        return 4
    if x=='VS1':
        return 5
    if x=='VVS2':
        return 6
    if x=='VVS1':
        return 7
    if x=='IF':
        return 8

In [604]:
df.clarity=df.clarity.apply(clarity)

In [605]:
df.cut.value_counts()

Ideal        16193
Premium      10301
Very Good     9030
Good          3710
Fair          1221
Name: cut, dtype: int64

In [606]:
def cut(x):
    if x=='Fair':
        return 5
    if x=='Good':
        return 4
    if x=='Very Good':
        return 3
    if x=='Premium':
        return 2
    if x=='Ideal':
        return 1


In [607]:
df.cut= df.cut.apply(cut)

In [608]:
df.color.unique()

array(['G', 'H', 'F', 'J', 'E', 'I', 'D'], dtype=object)

In [609]:
def color(x):
    if x=='J':
        return 7
    if x=='I':
        return 6
    if x=='H':
        return 5
    if x=='G':
        return 4
    if x=='F':
        return 3
    if x=='E':
        return 2
    if x=='D':
        return 1

In [610]:
df.color=df.color.apply(color)

In [611]:
df

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,1.14,1,4,6,61.0,56.0,6.74,6.76,4.12,9013
1,0.76,1,5,4,62.7,57.0,5.86,5.82,3.66,2692
2,0.84,1,4,5,61.4,56.0,6.04,6.15,3.74,4372
3,1.55,1,5,5,62.0,57.0,7.37,7.43,4.59,13665
4,0.30,1,4,2,61.9,57.0,4.28,4.31,2.66,422
...,...,...,...,...,...,...,...,...,...,...
40450,0.40,1,2,7,60.8,57.0,4.79,4.82,2.92,1278
40451,0.52,1,7,2,61.6,55.0,5.16,5.19,3.19,924
40452,1.00,3,2,3,59.4,63.0,6.47,6.42,3.83,5322


In [618]:
df= df.drop(['x'], axis=1)

In [619]:
X = df.drop('price', axis=1)
y = df['price']

In [620]:
modelo=LinReg()

In [621]:
X_train, X_test, y_train, y_test = tts(X,y, test_size=0.2)

In [622]:
modelo.fit(X,y)

LinearRegression()

In [623]:
y_pred = modelo.predict(X_test)

In [624]:
RMSE = (metrics.mean_squared_error(y_test, y_pred)) ** 0.5
RMSE

1207.8034701460283

In [392]:
models={
    'ridge': Ridge(),
    'lasso': Lasso(),
    'sgd': SGDRegressor(),
    'knn': KNeighborsRegressor(),
    'grad': GradientBoostingRegressor(),
}

In [393]:
for name, model in models.items():
    print("ENTRENANDO: ", name)
    model.fit(X_train, y_train)

ENTRENANDO:  ridge
ENTRENANDO:  lasso
ENTRENANDO:  sgd
ENTRENANDO:  knn
ENTRENANDO:  grad


In [394]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"------{name}------")    
    print('RMSE - ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

------ridge------
RMSE -  1205.267533320237
------lasso------
RMSE -  1205.9125697664094
------sgd------
RMSE -  4886133.749063441
------knn------
RMSE -  901.1615419894733
------grad------
RMSE -  613.8575195508364


In [437]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()
scaler = StandardScaler()

In [438]:
df2=df

In [439]:
df2["table"] = min_max.fit_transform(df2["table"].values.reshape(-1,1))

In [407]:
df2["depth"] = scaler.fit_transform(df2["depth"].values.reshape(-1,1))

In [440]:
X = df2.drop('price', axis=1)
y = df2['price']
modelo=LinReg()
X_train, X_test, y_train, y_test = tts(X,y, test_size=0.2)
modelo.fit(X,y)
y_pred = modelo.predict(X_test)

In [441]:
print('RMSE - Raiz Error Cuadratico Medio', metrics.mean_squared_error(y_test, y_pred)**0.5)

RMSE - Raiz Error Cuadratico Medio 1198.7741117956894


In [442]:
models={
    'ridge': Ridge(),
    'lasso': Lasso(),
    'sgd': SGDRegressor(),
    'knn': KNeighborsRegressor(),
    'grad': GradientBoostingRegressor(),
}

In [443]:
for name, model in models.items():
    print("ENTRENANDO: ", name)
    model.fit(X_train, y_train)

ENTRENANDO:  ridge
ENTRENANDO:  lasso


/home/eduardo/anaconda3/envs/Ironhack/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.051e+07, tolerance: 5.150e+07
  model = cd_fast.enet_coordinate_descent(


ENTRENANDO:  sgd
ENTRENANDO:  knn
ENTRENANDO:  grad


In [444]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"------{name}------")    
    print('RMSE - ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

------ridge------
RMSE -  1227.1161729570547
------lasso------
RMSE -  1209.3655685954377
------sgd------
RMSE -  1222.7857185514092
------knn------
RMSE -  715.1644873107989
------grad------
RMSE -  628.5837981505736


In [446]:
X

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,1.14,1,4,6,-0.520528,0.250000,6.74,6.76,4.12
1,0.76,1,5,4,0.667584,0.269231,5.86,5.82,3.66
2,0.84,1,4,5,-0.240972,0.250000,6.04,6.15,3.74
3,1.55,1,5,5,0.178361,0.269231,7.37,7.43,4.59
4,0.30,1,4,2,0.108473,0.269231,4.28,4.31,2.66
...,...,...,...,...,...,...,...,...,...
40450,0.40,1,2,7,-0.660306,0.269231,4.79,4.82,2.92
40451,0.52,1,7,2,-0.101194,0.230769,5.16,5.19,3.19
40452,1.00,3,2,3,-1.638750,0.384615,6.47,6.42,3.83


In [629]:
reg = RandomForestRegressor(n_estimators=6, max_depth=None, min_samples_leaf=2)
selector = RFECV(reg, step=1, cv=4)

In [638]:
selector.fit(X, y)

/home/eduardo/anaconda3/envs/Ironhack/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  warnings.warn(


RFECV(cv=4, estimator=RandomForestRegressor(min_samples_leaf=2, n_estimators=6))

In [631]:
predictors = pd.Series(X.columns)[selector.support_.tolist()].tolist()

In [632]:
X2 = df[predictors]
y2 = df["price"]

In [633]:
X_train, X_test, y_train, y_test = tts(X2, y2, test_size=0.2)

In [634]:
reg.fit(X_train, y_train)

RandomForestRegressor(min_samples_leaf=2, n_estimators=6)

In [636]:
rmse = np.sqrt(metrics.mean_squared_error(y_test, reg.predict(X_test)))

In [637]:
rmse

592.7197988656293

In [595]:
df.to_csv('diamonds.csv', index = False)